In [434]:
from jupyterthemes import jtplot
# onedork | grade3 | oceans16 | chesterish | monokai | solarizedl | solarizedd
jtplot.style(theme='grade3')

## EXAMEN NLP :
### Sujet : classifieur de commentaire produit.
#### Produit : Les cinq blessures qui empêchent d'être soi-même.
#### lien : https://www.amazon.fr/cinq-blessures-emp%C3%AAchent-d%C3%AAtre-soi-m%C3%AAme/dp/2266229486/ref=cm_cr_arp_d_product_top?ie=UTF8

#### Année : 2020 - 2021

#### Présenté par :
| Prénoms       |     Nom         |   
| ------------- |: -------------: |
| Aboubacar Sidiki        |        SIDIBE        |

GROUPE GEMA / IA-SCHOOL

## 1) Collection des commentaires produits d’amazone

### Procédures de scrapping
- a) Récupérer les commentaires positifs
- b) Récupérer les commentaires négatifs
- c) Concatener les deux datasets (commentaires positifs + commentaires négatifs)

In [161]:
!pip install unidecode

In [165]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import pickle

In [2]:
#### en-tête html utilisée pour la coordination entre le client (navigateur) et le serveur de open food fact, simule le comportement d'un navigateur
headers = {"Accept": "image/webp,*/*",
			"Accept-Encoding": "gzip, deflate, br",
			"Accept-Language": "fr,fr-FR;q=0.8,en-US;q=0.5,en;q=0.3",
			"Connection": "keep-alive",
			"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0"
		}

In [26]:
def parse_string(value):
    """
    Parse la chaine en supprimant \n \t \r
    
    :param value : la valeur de la chaine à parser
    :type value : String
    
    :return : la chaine parsée
    :rtype : String
    """
    if value is np.nan:
        return value
    else:
        chaine = value.replace('\n', ',')
        chaine = " ".join(chaine.split()).rstrip(',').strip(',').strip()
        return chaine

##### a. les commentaires positifs

In [39]:
data_comment_positif = []
for i in range(83):
    try:
        comment_url_positive = 'https://www.amazon.fr/cinq-blessures-emp%C3%AAchent-d%C3%AAtre-soi-m%C3%AAme/product-reviews/2266229486/ref=cm_cr_arp_d_viewpnt_lft?ie=UTF8&reviewerType=all_reviews&filterByStar=positive&pageNumber={}'.format(str(i))
        res = requests.get(comment_url_positive, headers = headers)
        soup = BeautifulSoup(res.text, 'html.parser')
        content = soup.findAll('div', attrs={'data-hook':'review'})
        for j in range(len(content)):
            note = content[j].findAll('a', attrs={'class':'a-link-normal'})[0].text.split(',')[0]
            label = parse_string(content[j].findAll('a', attrs={'class':'a-link-normal'})[1].text)
            comment = content[j].findAll('span',attrs={"data-hook":"review-body"})[0].text.strip()
            data_comment_positif.append([comment, label, note])
    except:
        continue     

In [41]:
len(data_comment_positif) #### nombre de commentaires positifs obtenus

830

In [45]:
df_pos = pd.DataFrame(data_comment_positif, columns=['commentaire', 'label', 'note'])

In [46]:
df_pos.shape

(830, 3)

In [53]:
df_pos.head()

,commentaire,label,note
0,"Lise Bourbeau, l'auteur de ces Cinq blessures ...",J'apprécie,4
1,Ce livre est une révélation. Je n’au Jamais pu...,Impressionnant,5
2,livre parfait pour commencer à se connaître et...,super,5
3,Si effectivement vous n’avez aucune connaissan...,Connaître concept Karmique pr saisir l’essence...,4
4,Alors ok il est vrai que je me suis reconnue d...,Hum ... sceptique,4


In [56]:
df_pos.to_csv('comment_positive.csv', index=False, encoding='utf-8')

##### b. les commentaires negatifs

In [43]:
data_comment_negatif = []
for i in range(28):
    try:
        comment_url_negatif = 'https://www.amazon.fr/cinq-blessures-emp%C3%AAchent-d%C3%AAtre-soi-m%C3%AAme/product-reviews/2266229486/ref=cm_cr_arp_d_viewpnt_rgt?ie=UTF8&reviewerType=all_reviews&filterByStar=critical&pageNumber={}'.format(str(i))
        res = requests.get(comment_url_negatif, headers = headers)
        soup = BeautifulSoup(res.text, 'html.parser')
        content = soup.findAll('div', attrs={'data-hook':'review'})
        for j in range(len(content)):
            note = content[j].findAll('a', attrs={'class':'a-link-normal'})[0].text.split(',')[0]
            label = parse_string(content[j].findAll('a', attrs={'class':'a-link-normal'})[1].text)
            comment = content[j].findAll('span',attrs={"data-hook":"review-body"})[0].text.strip()
            data_comment_negatif.append([comment, label, note])
    except:
        continue   

In [44]:
len(data_comment_negatif) #### nombre de commentaires négatifs obtenus

280

In [50]:
df_neg = pd.DataFrame(data_comment_negatif, columns=['commentaire', 'label', 'note'])

In [51]:
df_neg.shape

(280, 3)

In [52]:
df_neg.head()

,commentaire,label,note
0,"Franchement, je me suis arrêté au chapitre 1. ...","Si vos proches sont toxiques, ne lisez sous au...",1
1,Un livre à l'américaine qui fait très secte ! ...,Dramatique...,1
2,"Un peu déçue...Lise Bourbeau décrit les ""5 ble...",Déçue...,2
3,Le contenu de ce livre est complètement délira...,N’impsorte quoi,1
4,J'ai trouvé le thème du livre intéressant mais...,Pas convainquant,1


In [55]:
df_neg.to_csv('comment_negative.csv', index=False, encoding='utf-8')

##### c. Concatenation des deux datasets (commentaires positifs + commentaires négatifs)

In [251]:
df = pd.concat([df_pos, df_neg], ignore_index=True) #### concatenation des deux datasets

In [252]:
df.to_csv('comments.csv', index=False, encoding='utf-8') #### sauvegarde du dataset final

## 2. Nettoyage & &Featurisation des données

### 2.1. Analyse exploiratoire et nettoyage des données

In [253]:
df = pd.read_csv('comments.csv',encoding="utf-8")

#### a. Analyse exploiratoire

In [254]:
df.shape

(1110, 3)

In [255]:
df.head()

,commentaire,label,note
0,"Lise Bourbeau, l'auteur de ces Cinq blessures ...",J'apprécie,4
1,Ce livre est une révélation. Je n’au Jamais pu...,Impressionnant,5
2,livre parfait pour commencer à se connaître et...,super,5
3,Si effectivement vous n’avez aucune connaissan...,Connaître concept Karmique pr saisir l’essence...,4
4,Alors ok il est vrai que je me suis reconnue d...,Hum ... sceptique,4


In [256]:
df.describe()

,commentaire,label,note
count,1110,1110,1110
unique,1058,786,6
top,Intéressant,Signaler un abus,5
freq,5,41,605


In [257]:
len(df[789:830])

41

In [258]:
df = df.drop(labels=range(789, 830), axis=0)

In [259]:
df.shape

(1069, 3)

In [260]:
df['note'] = pd.to_numeric(df['note'])

In [261]:
len(df[df['note']>=4]) #### nombre de commentaires positives

789

In [262]:
len(df[df['note']<4]) #### nombre de commentaires negatifs

280

In [263]:
df.columns

Index(['commentaire', 'label', 'note'], dtype='object')

In [264]:
df['note'].value_counts()

5    605
4    184
1    125
3     95
2     60
Name: note, dtype: int64

#### b. Nettoyage

#### b.1) Macro nettoyage

- Enlever les duplicats
- Enlever les textes < seuil_mot
- Equilibrer le corpus (postif / négatif)

In [265]:
print(df.shape)
df.drop_duplicates(subset=['commentaire'],inplace=True)
df.shape

(1069, 3)


(1022, 3)

In [266]:
seuil_mot=15
def nb_mot(texte):
    return len(texte.split(' '))

df['Longueur']=df['commentaire'].apply(nb_mot)
df=df[df['Longueur']>seuil_mot]

In [267]:
# Binarisation 0 (1,2,3) / 1 (4,5)
df.loc[df[df['note'].astype(int)<4].index,'note']=0
df.loc[df[df['note'].astype(int)>3].index,'note']=1

# Je prends autant de texte positif que negatif (don j'identifie le label le moins present)
m=df['note'].value_counts().min()
Comments=pd.concat([df[df['note'].astype(int)==1].sample(m),df[df['note'].astype(int)==0].sample(m)])

In [268]:
Comments['note'].value_counts()

1    250
0    250
Name: note, dtype: int64

#### b.2) Nettoyage spécifique pour de l’analyse de texte

- Stop words
- carcteres speciaux (dont ponctuation)
- normalisation (accents et minuscule)

In [269]:
def nettoyage(texte):
    #charge la liste de stop words
    with open('stop-w_fr.txt') as f:
        s_w=list(set(unidecode(f.read()).split('\n')))
    s_w.append("possible")
    s_w.append("être")
    s_w.append("faire")
    
    # Chargement du lem
    lem=pickle.load(open('lemmatize_fr.pickle','rb'))
    lem['abonnement']="abonné"
    lem['abonner']="abonné"
    lem['abimes']='abime'
    
    tmp=[]
    # Normalisation : accents et minuscule
    texte=unidecode(texte.lower())
    p='[a-z]{2,}'
    final=''
    for mot in re.findall(p,texte):
        if mot not in s_w :
            try:
                if lem[mot.lower()] not in s_w:
                    final=final.strip()+' '+lem[mot.lower()]
            except:
                final=final.strip()+' '+mot.lower()
    return final

In [270]:
nettoyage("élément à modifier chez Mr Bean")

'element modifier mr bean'

In [271]:
nettoyage("Salut-à'tous les terriens des grandes visons simplifiées , ça va là et Coùcoù!")

'salut terrien grand vison simplifiees ca coucou'

In [272]:
negatif=[nettoyage(elem) for elem in Comments[Comments['note']==0]['commentaire']]
positif=[nettoyage(elem) for elem in Comments[Comments['note']==1]['commentaire']]

## 2.2. Featurisation des données

### a. TFIDF

In [273]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

#### negatif feature

In [274]:
X_neg = vectorizer.fit_transform(negatif)
print(vectorizer.get_feature_names())

print(X_neg.shape)

['abandonner', 'aberration', 'abime', 'abject', 'aborder', 'aboutir', 'absolu', 'abstenir', 'abstraction', 'absurde', 'absurdite', 'absurdites', 'abus', 'abusif', 'acabit', 'accables', 'acceder', 'acceptable', 'acceptation', 'accepter', 'accessible', 'accompagner', 'accord', 'accorder', 'accrocher', 'accumulation', 'achat', 'achete', 'acheter', 'acquérir', 'actif', 'activement', 'actuel', 'actuellement', 'adaptation', 'adepte', 'adequat', 'adhere', 'adherer', 'adherera', 'adherez', 'adheriez', 'admettre', 'adorer', 'adresser', 'adulte', 'affirmatif', 'affirmation', 'affirmer', 'affliger', 'agacant', 'agacer', 'age', 'agir', 'agreable', 'agrementee', 'aguerrir', 'aider', 'aimer', 'air', 'aisee', 'aiteur', 'ajouter', 'alexander', 'algorithme', 'alice', 'allechantes', 'allee', 'aller', 'allier', 'allusion', 'alzeihmer', 'amas', 'amazon', 'ame', 'ameliorer', 'americain', 'americaine', 'americains', 'ami', 'amour', 'amusant', 'amusee', 'amuser', 'an', 'analyser', 'ancien', 'aneries', 'anger

In [275]:
neg=pd.DataFrame(X_neg.toarray(),columns=vectorizer.get_feature_names())
neg

,abandonner,aberration,abime,abject,aborder,aboutir,absolu,abstenir,abstraction,absurde,...,vrai,vraiment,vraisemblance,vronchotes,vue,wc,wtf,youtube,zero,zeste
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.254916,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.264087,0.0,0.0,0.0,0.0
247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.243561,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


#### positive feature

In [276]:
X_pos = vectorizer.fit_transform(positif)
print(vectorizer.get_feature_names())

print(X_pos.shape)

['abandon', 'abandonner', 'abime', 'abordable', 'aborder', 'aboutir', 'absence', 'abstenir', 'abstraction', 'acceptation', 'acceptee', 'accepter', 'acces', 'accessible', 'accrocher', 'accueillir', 'accumuler', 'achat', 'achete', 'acheter', 'acquerir', 'acquérir', 'acter', 'action', 'adapter', 'adepte', 'adhere', 'admettre', 'adonner', 'adorer', 'adresser', 'adulte', 'agacant', 'agacantes', 'age', 'agir', 'agissement', 'agreable', 'agreablement', 'aidee', 'aider', 'aimer', 'aimezsi', 'aisement', 'alie', 'alimentaire', 'aller', 'allumer', 'alterent', 'amateur', 'amazon', 'ambition', 'ame', 'ameliorer', 'amene', 'amener', 'amenes', 'americaine', 'ames', 'ami', 'amour', 'amuser', 'an', 'analysee', 'analyser', 'analytique', 'ancien', 'and', 'angle', 'annees', 'annoncer', 'anxieux', 'apercu', 'apparait', 'apparence', 'appel', 'appliquer', 'apport', 'apporter', 'appreciable', 'apprecie', 'apprecier', 'apprehender', 'apprendre', 'appris', 'approcher', 'approfondir', 'approfondissement', 'appuy

In [277]:
pos=pd.DataFrame(X_pos.toarray(),columns=vectorizer.get_feature_names())
pos

,abandon,abandonner,abime,abordable,aborder,aboutir,absence,abstenir,abstraction,acceptation,...,voltaire,vouloir,voulu,vrai,vraiment,vue,vulgarise,wahou,web,wow
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
246,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
247,0.152802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
248,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.211608,0.0,0.0,0.0


### b. Word2vec

#### Analyse du corpus

In [278]:
### corpus
Comments.shape

(500, 4)

In [279]:
Comments.head()

,commentaire,label,note,Longueur
312,Livre très intéressant et enrichissant pour qu...,À lire absolument et à recommander,1,20
632,rapide et conforme à ma demanderien à signaler...,parfait,1,19
250,Il faut dépasser le côté ésotérique et se plon...,A lire absoluement,1,32
264,"Ce livre est très intéressant à lire, malgré l...",Très bien,1,37
442,cet ouvrage pose les bases des blessures de l'...,pour soigner ses blessures,1,64


In [280]:
Comments['note'].value_counts()

1    250
0    250
Name: note, dtype: int64

In [281]:
Comments.columns

Index(['commentaire', 'label', 'note', 'Longueur'], dtype='object')

In [282]:
for i in Comments.index:
    Comments['commentaire'][i] = nettoyage(Comments['commentaire'][i])

<ipython-input-282-24165bfb379f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Comments['commentaire'][i] = nettoyage(Comments['commentaire'][i])


In [215]:
##!pip install nltk

In [216]:
##import nltk
##nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

#### construction de la liste des phrases à partir du corpus

In [283]:
import nltk
list_sents = [nltk.word_tokenize(sent) for sent_tok in Comments['commentaire'] for sent in nltk.sent_tokenize(sent_tok)]

In [284]:
list_sents

[['livrer',
  'interessant',
  'enrichissant',
  'quelqu',
  'souhaiter',
  'eclairages',
  'propre',
  'conditionnement',
  'hesitez',
  'recommander'],
 ['rapide',
  'conformer',
  'demanderien',
  'signalertout',
  'okje',
  'recommander',
  'probleme',
  'carplus',
  'pratiquer',
  'deplacer'],
 ['falloir',
  'depasser',
  'coter',
  'esoterique',
  'plonger',
  'caracteristiques',
  'masquer',
  'bluffer',
  'appris',
  'entourage',
  'lire'],
 ['livrer',
  'interessant',
  'lire',
  'fauter',
  'francais',
  'apprendre',
  'choses',
  'petit',
  'deception',
  'decouvrir',
  'guerir',
  'falloir',
  'acheter',
  'deuxieme',
  'tome'],
 ['ouvrager',
  'poser',
  'baser',
  'blessure',
  'ame',
  'prendre',
  'conscience',
  'blessure',
  'aider',
  'accepter',
  'aider',
  'soigner',
  'debuter',
  'travail',
  'personnel',
  'livrer',
  'adapter',
  'interroger',
  'amene',
  'reflexion',
  'partir',
  'venir',
  'terminer',
  'devoir',
  'mettre',
  'pratiquer'],
 ['lire',
  're

In [286]:
!pip install gensim

In [310]:
from gensim.models import Word2Vec
word2vec_model = Word2Vec(sentences=list_sents, vector_size=80, window=12, min_count=3, workers=4)

In [314]:
##getting a word vector
word2vec_model.wv['livrer']

array([ 0.5070165 , -0.3282085 ,  0.40800244,  0.5609819 , -0.5225148 ,
        0.12045065,  0.23681185, -0.50316596, -0.52502817, -0.18754344,
        0.01847555, -0.53893316, -0.35471103,  0.10907334,  0.44925645,
        0.6957178 , -0.843368  ,  0.17444076, -0.3996922 , -0.39274618,
        0.7209371 ,  0.07917201,  0.3574111 , -0.12433771, -0.0581608 ,
       -0.18182538,  0.04351588,  0.36592075, -1.0237124 , -0.39706334,
        0.44850802, -0.00944066,  0.6671096 , -0.5799879 , -0.08917297,
        1.0609721 ,  1.1923076 , -0.29237574, -0.8458057 , -0.5719743 ,
       -0.21815783,  0.60279804, -0.97239375,  0.05643602,  0.7695424 ,
       -0.19593236, -0.32305104,  0.8741776 ,  0.618578  ,  0.61167777,
        0.38914013,  0.5812729 ,  0.1743936 ,  0.34995043,  0.37837237,
       -0.17188083,  0.13351655, -0.373595  , -0.45725623, -0.48882645,
       -0.32528535, -0.18086542, -0.00616943, -0.4122063 , -0.73108536,
       -0.87320536, -0.41836149,  0.97264254,  0.17442352,  0.95

In [315]:
##getting a word vector
word2vec_model.wv['lire']

array([ 0.29078588, -0.19115406,  0.23203765,  0.32040122, -0.29923567,
        0.0839181 ,  0.14512537, -0.31286284, -0.29909092, -0.10032471,
        0.00087996, -0.33004698, -0.20000999,  0.06833508,  0.27032617,
        0.40587997, -0.5114492 ,  0.09963319, -0.22456042, -0.23047408,
        0.41108796,  0.04752117,  0.21362147, -0.08313479, -0.03807902,
       -0.10815512,  0.02668329,  0.20032951, -0.59885675, -0.23899297,
        0.27328658, -0.00750132,  0.3915689 , -0.34885907, -0.05460777,
        0.634874  ,  0.70805395, -0.17252931, -0.51465523, -0.33082637,
       -0.12224069,  0.35848624, -0.5795918 ,  0.03516242,  0.46572968,
       -0.10660466, -0.19060564,  0.5153884 ,  0.35337278,  0.364625  ,
        0.23165455,  0.33736572,  0.11853718,  0.2080842 ,  0.22796807,
       -0.10653148,  0.08217599, -0.2038605 , -0.2795336 , -0.30326203,
       -0.20130563, -0.11346208, -0.00614768, -0.25015092, -0.42317712,
       -0.5264799 , -0.23772137,  0.5802335 ,  0.09762774,  0.57

In [316]:
##getting a word vector
word2vec_model.wv['formation']

array([ 6.96124136e-02, -5.46828397e-02,  4.63789515e-02,  8.30313712e-02,
       -7.13283643e-02,  2.13399660e-02,  3.30766253e-02, -7.93594643e-02,
       -6.64969385e-02, -2.39554942e-02, -6.49996288e-03, -7.85209835e-02,
       -4.46550995e-02,  1.12440307e-02,  5.11470512e-02,  1.00934841e-01,
       -9.88590568e-02,  2.72364039e-02, -5.36979623e-02, -5.52829467e-02,
        8.63402039e-02,  3.63742770e-03,  5.29478230e-02, -2.67270524e-02,
       -1.82577074e-02, -1.48563618e-02,  9.21815634e-03,  3.81692722e-02,
       -1.43111557e-01, -6.32716194e-02,  5.96327335e-02, -1.03938384e-02,
        8.48401412e-02, -7.05422089e-02, -4.12221206e-03,  1.29762724e-01,
        1.45869523e-01, -4.80494052e-02, -1.22025795e-01, -7.86944702e-02,
       -2.91689746e-02,  7.88030699e-02, -1.19937964e-01, -1.33673137e-04,
        1.08462930e-01, -2.92209759e-02, -4.67079990e-02,  1.05930597e-01,
        7.90711716e-02,  9.05489847e-02,  5.93065992e-02,  8.32259357e-02,
        1.93783063e-02,  

### b. Camembert (Le BERT français)

In [317]:
import torch
from transformers import CamembertTokenizer,CamembertModel

tokenizer = CamembertTokenizer.from_pretrained('camembert/camembert-large')
model = CamembertModel.from_pretrained('camembert/camembert-large',output_hidden_states=True)

In [328]:
# Extract the last layer's features
line = "J'aime le camembert !"
input_ids = tokenizer.encode(line, return_tensors='pt') 
outputs = model(input_ids)

In [329]:
input_ids.shape

torch.Size([1, 10])

In [330]:
outputs[2][-4:]

(tensor([[[ 0.1587,  0.3786,  0.0825,  ...,  0.1899,  0.0925, -0.2983],
          [ 0.4463, -0.0386, -0.4613,  ...,  0.3045, -0.0562, -0.3966],
          [ 0.0162,  0.4548,  0.4028,  ...,  0.3415,  0.1764, -0.1404],
          ...,
          [ 0.3938,  0.3159, -0.2385,  ...,  0.3451,  0.3870,  0.3684],
          [ 0.1471,  0.0064,  0.3660,  ...,  0.2415, -0.3712,  0.0727],
          [-0.0252, -0.0246,  0.0434,  ...,  0.0147, -0.0142, -0.0358]]],
        grad_fn=<NativeLayerNormBackward>),
 tensor([[[-0.0578,  0.4780,  0.3756,  ...,  0.2118,  0.1431,  0.1120],
          [ 0.2987, -0.2125, -0.5713,  ...,  0.2652, -0.0030, -0.1967],
          [ 0.3370,  0.4695,  0.3492,  ...,  0.1969,  0.0118, -0.1037],
          ...,
          [ 0.3618,  0.3188, -0.2318,  ...,  0.1946,  0.2531,  0.4184],
          [ 0.2292, -0.0217,  0.1935,  ...,  0.1682, -0.2649,  0.2151],
          [-0.0282, -0.0402,  0.0247,  ...,  0.0333,  0.0277, -0.0330]]],
        grad_fn=<NativeLayerNormBackward>),
 tensor([[[-0.

In [331]:
import pandas as pd
import numpy as np

l=[]
for i in range(-1,-5,-1):
    l.append(pd.DataFrame(np.mean(outputs[2][i].squeeze(0).detach().numpy(),axis=0)).T)
    
pd.concat(l,ignore_index=True).mean(axis=0)

0       0.078970
1       0.036869
2       0.041912
3       0.090438
4       0.144143
          ...   
1019   -0.028083
1020    0.049764
1021    0.029793
1022    0.010036
1023    0.017181
Length: 1024, dtype: float32

## 3. classifieur binaire

### 3.1. Methode TFIDF

In [334]:
### dataset
Comments['note'].value_counts()

1    250
0    250
Name: note, dtype: int64

In [335]:
from sklearn.model_selection import train_test_split

X = Comments['commentaire']
y = Comments['note']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

#### TfidfVectorizer

In [336]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train_final = vectorizer.fit_transform(X_train)

print(X_train_final.shape)

(350, 2051)


#### pipeline scikit-learn

In [338]:
from sklearn.svm import LinearSVC

clf = LinearSVC()
clf.fit(X_train_final, y_train)

LinearSVC()

#### une autre manière de pipelines pour résumer notre travail

In [339]:
from sklearn.pipeline import Pipeline

text_clf = Pipeline([('tfidf', TfidfVectorizer()),('clf', LinearSVC()),])
text_clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [340]:
predictions = text_clf.predict(X_test)

In [341]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.76
[[61 13]
 [23 53]]
              precision    recall  f1-score   support

           0       0.73      0.82      0.77        74
           1       0.80      0.70      0.75        76

    accuracy                           0.76       150
   macro avg       0.76      0.76      0.76       150
weighted avg       0.77      0.76      0.76       150



### 3.2. Word2Vec

In [346]:
import nltk
list_sents = [nltk.word_tokenize(sent) for sent_tok in X_train for sent in nltk.sent_tokenize(sent_tok)]

In [347]:
list_sents

[['lire',
  'relire',
  'moderationpenser',
  'prendre',
  'note',
  'debutc',
  'livrer',
  'personnel'],
 ['lu',
  'livrer',
  'proposer',
  'analyser',
  'comportement',
  'attitude',
  'corporel',
  'basees',
  'blessure',
  'ancien',
  'forgees',
  'primer',
  'enfance',
  'facile',
  'reconnaitre',
  'premiere',
  'lecture'],
 ['recommander',
  'livrer',
  'blessure',
  'enfance',
  'consequent',
  'masquer',
  'porter',
  'niveau',
  'physique',
  'reconnaitrez'],
 ['lise',
  'bourbeau',
  'contenter',
  'decrire',
  'blessure',
  'indiquer',
  'remedesen',
  'blessure',
  'memoires',
  'cellulaire',
  'falloir',
  'libererelles',
  'sytemendocrinien',
  'endocrino',
  'psychologie',
  'cerveau',
  'lightning',
  'process',
  'eft',
  'coder',
  'guerison',
  'complements'],
 ['branchee',
  'developpement',
  'personnel',
  'livrer',
  'generalites',
  'analyser',
  'percutant',
  'auteur',
  'expliquer',
  'morphologie',
  'traduire',
  'blessure',
  'trouver',
  'verifiables',

In [349]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=list_sents, vector_size=80, window=12, min_count=3, workers=8)

In [350]:
model.corpus_count

350

In [353]:
import numpy

def get_vect(word, model):
    try:
        return model.wv[word]
    except KeyError:
        return numpy.zeros((model.vector_size,))

def sum_vectors(phrase, model):
    return sum(get_vect(w, model) for w in phrase)

def word2vec_features(X, model):
    feats = numpy.vstack([sum_vectors(p, model) for p in X])
    return feats

wv_train_feat = word2vec_features(X_train, model)
wv_train_feat.shape

(350, 80)

In [374]:
wv_train_feat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [376]:
from sklearn.linear_model import LogisticRegression
clfwv = LogisticRegression(solver='lbfgs', max_iter=1000)
clfwv.fit(wv_train_feat, y_train)

LogisticRegression(max_iter=1000)

In [356]:
wv_test_feat = word2vec_features(X_test, model)

In [357]:
clfwv.score(wv_test_feat, y_test)

0.49333333333333335

In [377]:
wv_test_feat = word2vec_features(X_test, model)

In [378]:
y_pred = clfwv.predict(wv_test_feat)

In [380]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.49333333333333335
[[74  0]
 [76  0]]
              precision    recall  f1-score   support

           0       0.49      1.00      0.66        74
           1       0.00      0.00      0.00        76

    accuracy                           0.49       150
   macro avg       0.25      0.50      0.33       150
weighted avg       0.24      0.49      0.33       150



C:\Users\SIDIKI\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 3.3. CamenBERT

#### Nous commençons par installer transformers et sentencepiece, deux packages nécessaires à l’usage de Camembert.

In [400]:
#!pip install transformers
#!pip install sentencepiece

In [401]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

#### Chargement du modèle

In [413]:
reviews = Comments['commentaire']
labels = Comments['note']

In [402]:
# load model, tokenizer and weights
camembert, tokenizer, weights = (ppb.CamembertModel, ppb.CamembertTokenizer, 'camembert-base')

# Load pretrained model/tokenizer
tokenizer = tokenizer.from_pretrained(weights)
model = camembert.from_pretrained(weights)

#### on enleve les commentaires trop longs. Car Bert ne sait que tokéniser des phrases de longueur maximale de 512 tokens

In [408]:
# see if there are length > 512
max_len = 0
for i,sent in enumerate(reviews):
    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    if len(input_ids) > 512:
        print("annoying review at", i,"with length",
              len(input_ids))
    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  233


#### pas de phrases trouvées dont la longeur depasse la longueur max autorisée (512)

#### Padding

#### nous faisons un padding de 472 tokens pour homogénéiser la longueur de phrases. Cela rendra l’entraînement plus simple.

In [409]:
tokenized = reviews.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape

(500, 233)

In [410]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(500, 233)

#### Encodage du texte

#### nous transformons les tokens en tensor pour les passer dans le transformer. 
#### Seule la dernière couche est conservée pour faire la classification.

In [412]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

#### Entrainement d'un modèle logistique

#### Comme nous avons seulement besoin du premier token (CLS qui signifie classification) pour le modèle logistique, nous faisons en slice avec [:,0,:].

In [414]:
features = last_hidden_states[0][:,0,:].numpy()
labels = labels
labels

312     1
632     1
250     1
264     1
442     1
       ..
851     0
1070    0
865     0
874     0
998     0
Name: note, Length: 500, dtype: int64

In [415]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [416]:
# Grid search
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scrores: ', grid_search.best_score_)

best parameters:  {'C': 15.789557894736841}
best scrores:  0.7466666666666667


In [417]:
lr_clf = LogisticRegression(C=grid_search.best_params_['C'])
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=15.789557894736841)

In [418]:
y_pred = lr_clf.predict(test_features)

#### Evaluation du modèle

In [430]:
from sklearn import metrics
print("Accuracy:", metrics.accuracy_score(test_labels, y_pred))
print("Precision:", metrics.precision_score(test_labels, y_pred))
print("Recall:", metrics.recall_score(test_labels, y_pred))
print("F1-score:", metrics.f1_score(test_labels, y_pred))
print("MCC:", metrics.matthews_corrcoef(test_labels, y_pred))
print("Confusion matrix:", metrics.confusion_matrix(test_labels, y_pred))
print(metrics.classification_report(test_labels, y_pred))

Accuracy: 0.72
Precision: 0.6721311475409836
Recall: 0.7321428571428571
F1-score: 0.7008547008547008
MCC: 0.4400161323469335
Confusion matrix: [[49 20]
 [15 41]]
              precision    recall  f1-score   support

           0       0.77      0.71      0.74        69
           1       0.67      0.73      0.70        56

    accuracy                           0.72       125
   macro avg       0.72      0.72      0.72       125
weighted avg       0.72      0.72      0.72       125

